# **Real Estate: It's Ame-azing!**
### *A report on predicting the sales price of houses in Ames, Iowa.*
| | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|
|Emily K. Sanders| | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |Project 2: Ames Housing Prices|
|DSB-318| | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |April 19, 2024|
---

### **Appendix B:** Missing and invalid value explorations.

As described in the main notebook, I conducted thorough investigations into all missing and suspicious values in the dataset prior to partitioning it for training and testing, so as to maximize my familiarity with the types of values that are likely to be problems in future data.  This notebook details those investigations, and how I resolved each issue.  Although I fixed several individual values here, all of my actions were afterwards compiled into reproducible rules that can be applied to novel data without any of the hands-on work contained here.

After importing the relevant modules, I generated several summary statistics and descriptions, and reviewed them for signs missing data, inappropriate data types, or signs of incorrect data.  Once those preliminary checks were complete, I began to dig more deeply into particular columns.

In [2]:
# Import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Import the data
housing = pd.read_csv('./datasets/train.csv')

In [ ]:
# Check it out
housing.info()

None the data types were unreasonable, nor did they suggest any "messy" values "contaminating" entire columns.  As I expected, some of the columns were formatted as objects (strings) and would have to be converted to numeric types to be included in the model, and some columns that might be best used as categorical were formatted as integers. 

## Missing Values

Because of the size of this dataset, I opted to create a dataframe containing the information on null values, rather than just generating this report in output.  Furthermore, to make it easier to see all 81 features and make notes on them, I saved this dataframe as a csv, and opened it in Excel to make notes in the margins.  A "clean copy" of the csv, as generated in the cell below, as well as my annotated copy, are included in the supplementary materials.  Please note that the annotated copy was only used for puzzling out the missing values; readers who cannot decipher my shorthand and sentence fragments will not be missing out on any important information.  Any reader who wishes to see the orignal output can do so by uncommenting the code snippet below, but for the reader's ease, a table of missing values is included.

In [ ]:
# Generate a report of NAs and export it to CSV
housing_nas = pd.DataFrame(housing.isna().sum())
housing_nas.columns = ['NAs_count']
housing_nas = housing_nas.sort_values(by = 'NAs_count', ascending = False)
housing_nas.to_csv('./output/nas_count.csv')

In [ ]:
# Uncomment and run this code to see the dataframe within the report.
#housing_nas

|      Feature      |Null Count|    Feature    |Null Count|    Feature   |Null Count|
|----------------------|----|----------------------|----|----------------------|----|
|basemt_condition      |  55|garage_condition      | 114|masonry_type          |1240|
|basemt_quality        |  55|garage_quality        | 114|masonry_area          |  22|
|basemt_exposure       |  58|garage_type           | 113|pool_quality          |2042|
|basemt_fin_1_qual     |  55|garage_year_built     | 114|fireplace_quality     |1000|
|basemt_fin_2_qual     |  56|garage_finish         | 114|misc_feature          |1986|
|basemt_fin_1_sf       |   1|garage_area           |   1|fence                 |1651|
|basemt_fin_2_sf       |   1|garage_cars           |   1|alley                 |1911|
|basemt_unf_sf         |   1|basemt_half_bath      |   2|lot_frontage          | 330|
|basemt_total_sf       |   1|basemt_full_bath      |   2|**All Other Features**|   0|

Because the [received data dictionary](https://www.kaggle.com/competitions/318-ames-competition/data) says that NA was used to indicate houses that simply did not have a given element, I deemed it likely that any rows for which *all* of the basement-related columns were NA (or 0, where applicable) represented houses that simply did not have basements.  If the hypothesis that most of these NAs are tied to such houses were correct, then they would all be contained within the subset of the dataframe defined by isolating the basement-related feature with the greatest number of NAs.

In [ ]:
# Bsmt Exposure has the most NAs of the Bsmt categories
basements = housing[housing['basemt_exposure'].isnull()]
basements.isnull().sum()

In [ ]:
# Print how many NAs of each of these columns are contained within the subset, out of total NAs for the column
print(f"Nulls contained within the rows where Exposure is null ({basements['basemt_exposure'].isnull().sum()}/58)")
print(f"1. Condition: {basements['basemt_cond'].isnull().sum()}/55")
print(f"2. Quality: {basements['basemt_qual'].isnull().sum()}/55")
print(f"3. Finished 1 Type: {basements['basemt_fin_1_qual'].isnull().sum()}/55")
print(f"4. Finished 2 Type: {basements['basemt_fin_2_qual'].isnull().sum()}/56")
print(f"5. Half baths: {basements['basemt_half_bath'].isnull().sum()}/2")
print(f"6. Full baths: {basements['basemt_full_bath'].isnull().sum()}/2")
print(f"7. Unfinished Sqft: {basements['basemt_unf_sf'].isnull().sum()}/1")
print(f"8. Finished 1 Sqft: {basements['basemt_fin_1_sf'].isnull().sum()}/1")
print(f"9. Finished 2 Sqft: {basements['basemt_fin_2_sf'].isnull().sum()}/1")
print(f"10. Total Sqft: {basements['basemt_total_sf'].isnull().sum()}/1")

All but one of these NAs were contained within the subsetted dataframe.  Notably, the descriptive columns very consistently had 55 NAs, and the quantitative columns - for which 0 is a valid, non-NA entry - had much fewer.  Below I have listed the number of 0s in these quantitative columns.

In [ ]:
print("Furthermore, the number of 0s in these columns are:")
print(f"5. Half baths: {len(basements[basements['basemt_half_bath']==0])}")
print(f"6. Full baths: {len(basements[basements['basemt_full_bath']==0])}")
print(f"7. Unfinished Sqft: {len(basements[basements['basemt_unf_sf']==0])}")
print(f"8. Finished 1 Sqft: {len(basements[basements['basemt_fin_1_sf']==0])}")
print(f"9. Finished 2 Sqft: {len(basements[basements['basemt_fin_2_sf']==0])}")
print(f"10. Total Sqft: {len(basements[basements['basemt_total_sf']==0])}")

For every one of these features, the sum of the NAs and the 0s was equal to or greater than 55.  It is completely plausible that someone could have a basement without any bathrooms or finished space in it - indeed, I was surprised those columns did not have *more* 0s - so the columns with more than 55 NAs or 0s were of no concern.  The fact that no columns had *fewer* than 55 NAs or 0s - i.e., the columns were consistent "no"s all across the way - indicated that these "missing" values were only missing a readable format; they were not missing *information*.  They showed that 55 of the houses in the dataset did not have basements.

The only remaining mysteries in this section were the single NA in *basemt_finished_type_2* and the 3 NAs in *basemt_exposure* that exist outside of these core 55.  Because this was at most 4 rows, I subsetted down to them and inspected them visually.

In [ ]:
# Exclude those 55 houses with no basements
basements2 = housing[housing['basemt_fin_1_qual'].notnull()]
# Isolate the remaining null values
basements2 = basements2[(basements2['basemt_fin_2_qual'].isnull()) | (basements2['basemt_exposure'].isnull())]
basements2

In [ ]:
# For context
print(housing['basemt_exposure'].value_counts(dropna = False))
print(housing['basemt_fin_2_qual'].value_counts(dropna = False))

All 4 of these rows had populated, plausible values throughout the other basement-related columns, and did not present any obvious data integrity concerns elsewhere.  I concluded that these NAs were errors.

Because so many houses with basements had a "No" in the *bsmt_exposure* column - which makes sense, given that basements are usually underground - I was willing to assume that that was the intended value for the 3 NAs in rows that otherwise clearly indicate the presence of a basement, and changed them to "No."  The 1 remaining NA in the *basemt_finished_type_2* was harder to understand.  With no real clues to go on, I changed this to "not-present" along with all the other NAs in this column, even though the row indicated that the house did have a basement.  Although that imputation is technically incorrect, it was likely the best solution to the problem.  Whatever the true information is, it was unavailable, and thus of no more use in the model than if it did not exist at all.  That house got "credit" in the model for having a basement via its other, correctly populated features, and did not receive "unearned credit" through incorrect imputation here.

For the quantitative columns, I replaced the NAs with 0s.  For the qualitative columns, I replaced the NAs with "not-present" to make later dummy coding easier.

In [ ]:
# Changing those 3 exposure NAs with other populated features to "No"
housing.loc[((housing['basemt_exposure'].isna()) & (housing['basemt_cond'].notna())), 'basemt_exposure'] = 'No'
print('There were previously 1339 "No"s.')
print(housing['basemt_exposure'].value_counts(dropna = False))

In [ ]:
# Changing rest of the NAs
housing['basemt_exposure'] = housing['basemt_exposure'].fillna('not-present')
housing['basemt_cond'] = housing['basemt_cond'].fillna('not-present')
housing['basemt_qual'] = housing['basemt_qual'].fillna('not-present')
housing['basemt_fin_1_qual'] = housing['basemt_fin_1_qual'].fillna('not-present')
housing['basemt_fin_2_qual'] = housing['basemt_fin_2_qual'].fillna('not-present')
housing['basemt_half_bath'] = housing['basemt_half_bath'].fillna(0)
housing['basemt_full_bath'] = housing['basemt_full_bath'].fillna(0)
housing['basemt_unf_sf'] = housing['basemt_unf_sf'].fillna(0)
housing['basemt_fin_1_sf'] = housing['basemt_fin_1_sf'].fillna(0)
housing['basemt_fin_2_sf'] = housing['basemt_fin_2_sf'].fillna(0)
housing['basemt_total_sf'] = housing['basemt_total_sf'].fillna(0)

In [ ]:
# Rechecking the null counts
print(f"0. Exposure: {housing['basemt_exposure'].isnull().sum()}/58)")
print(f"1. Condition: {housing['basemt_cond'].isnull().sum()}/55")
print(f"2. Quality: {housing['basemt_qual'].isnull().sum()}/55")
print(f"3. Finished 1 Type: {housing['basemt_fin_1_qual'].isnull().sum()}/55")
print(f"4. Finished 2 Type: {housing['basemt_fin_2_qual'].isnull().sum()}/56")
print(f"5. Half baths: {housing['basemt_half_bath'].isnull().sum()}/2")
print(f"6. Full baths: {housing['basemt_full_bath'].isnull().sum()}/2")
print(f"7. Unfinished Sqft: {housing['basemt_unf_sf'].isnull().sum()}/1")
print(f"8. Finished 1 Sqft: {housing['basemt_fin_1_sf'].isnull().sum()}/1")
print(f"9. Finished 2 Sqft: {housing['basemt_fin_2_sf'].isnull().sum()}/1")
print(f"10. Total Sqft: {housing['basemt_total_sf'].isnull().sum()}/1")

In [ ]:
# Rechecking the 0 counts
print("Furthermore, the number of 0s in these columns are:")
print(f"5. Half baths: {len(housing[housing['basemt_half_bath']==0])}, from 1923")
print(f"6. Full baths: {len(housing[housing['basemt_full_bath']==0])}, from 1200")
print(f"7. Unfinished Sqft: {len(housing[housing['basemt_unf_sf']==0])}, from 165")
print(f"8. Finished 1 Sqft: {len(housing[housing['basemt_fin_1_sf']==0])}, from 657")
print(f"9. Finished 2 Sqft: {len(housing[housing['basemt_fin_2_sf']==0])}, from 1803")
print(f"10. Total Sqft: {len(housing[housing['basemt_total_sf']==0])}, from 54")

Displayed below is a very similar analysis for the garage-related features, which were also noted in the received data dictionary to use NA as a marker for not having a garage.  For brevity, because this analysis is so similar to the basements section, each step is accompanied by much less, if any, explanation.

In [ ]:
# Garage Condition has the maximum number of NAs in the garage columns (not exclusively)
garages = housing[housing['garage_cond'].isnull()]
garages.isnull().sum()

In [ ]:
# Print how many NAs of each of these columns are contained within the subset, out of total NAs for the column
print(f"Nulls contained within the rows where Condition is null ({garages['garage_cond'].isnull().sum()}/114)")
print(f"1. Quality: {garages['garage_qual'].isnull().sum()}/114")
print(f"2. Type: {garages['garage_type'].isnull().sum()}/113")
print(f"3. Year Built: {garages['garage_yr_blt'].isnull().sum()}/114")
print(f"4. Finish: {garages['garage_finish'].isnull().sum()}/114")
print(f"5. Area: {garages['garage_area'].isnull().sum()}/1")
print(f"6. Cars: {garages['garage_cars'].isnull().sum()}/1")

In [ ]:
print("Furthermore, the number of 0s in these columns are:")
print(f"5. Area {len(garages[garages['garage_area']==0])}")
print(f"6. Cars {len(garages[garages['garage_cars']==0])}")

Every single one of these NAs was contained within the same subset of rows, and displayed the same pattern as the basements: the descriptive features had high, consistent NA counts, and the quatitative features, for which 0 is a valid entry, had many fewer NAs.  Indeed, these features had exactly enough 0s to match the NAs in the descriptive features.  Unlike *basemt_half_bath*, for example, which may be 0 even in a house with a basement, it is not possible for a garage to exist but have no area, so if my interpretation of these NAs and 0s was correct, these columns should not have had 0s in any rows that had other populated garage features.  (Although a homeowner could conceivably use the word "garage" to refer to a space not large enough for even a single car, it seems likely that such a structure would be represented in the dataset as *misc_feature - shed*, rather than as a garage.) 

In [ ]:
print("Furthermore, the TOTAL number of 0s in these columns are:")
print(f"5. Area {len(housing[housing['garage_area']==0])}")
print(f"6. Cars {len(housing[housing['garage_cars']==0])}")

As predicted, these columns had no 0s except for the ones in the rows of interest.

The final step before concluding that these houses simply do not have garages was to investigate why *garage_type* is populated in a row where none of the others were.

In [ ]:
test_row = garages[garages['garage_type'].notnull()]
test_row

This row did not show signs of overall corruption, which made the option of simply dropping it unappealing.  However, the *garage_type* column, which indicated a detached garage, stood in contrast to every other indicator, which pointed towards this house not having a garage at all.  There was risk of introducing error no matter what I did with this value.  

To get a sense of how this incongruous value may have come to be, I dug deeper into the [provenance of the dataset](https://jse.amstat.org/v19n3/decock.pdf): the records system in the Ames City Assessor's Office (de Cock, 2011).  This narrowed the possibilities.  While volunteer respondents may misrepresent or omit data for any number of reasons, a government office tasked with assessing taxes on the populace is much less likely to engage in mischievous reporting or fatigue attrition.  Therefore, I concluded that this discrepancy was more likely to be a mistake than to be an incomplete but otherwise accurate report.  The fact that this was the same row where *garage_cars* and *garage_area* were marked NA, rather than 0 like in all other relevant rows, further supported my hypothesis that there was something wrong with it, rather than the idea that it was simply missing some information.  Therefore, I assumed that the house did not have a garage, this value was recorded erroneously, and thus, to remove it.  I did this by specifying the exact row for the data at hand, but wrote a cleaning code at the end of my investigations to generalize it to future data without the need for hands-on intervention.

In [ ]:
print(housing.loc[housing['pid']==910201180, 'garage_type'])
housing.loc[housing['pid']==910201180, 'garage_type'] = np.nan
print(housing.loc[housing['pid']==910201180, 'garage_type'])
print(housing['garage_type'].isnull().sum())

With that, the mystery of the garage-related NAs was solved, and I corrected the values in the same way as with the basements.

In [ ]:
# Changing rest of the NAs
housing['garage_cond'] = housing['garage_cond'].fillna('not-present')
housing['garage_qual'] = housing['garage_qual'].fillna('not-present')
housing['garage_type'] = housing['garage_type'].fillna('not-present')
housing['garage_yr_blt'] = housing['garage_yr_blt'].fillna('not-present')
housing['garage_finish'] = housing['garage_finish'].fillna('not-present')
housing['garage_area'] = housing['garage_area'].fillna(0)
housing['garage_cars'] = housing['garage_cars'].fillna(0)

In [ ]:
# Rechecking the NA counts
print(f"0. Condition: {housing['garage_cond'].isnull().sum()}/114")
print(f"1. Quality: {housing['garage_qual'].isnull().sum()}/114")
print(f"2. Type: {housing['garage_type'].isnull().sum()}/113")
print(f"3. Year Built: {housing['garage_yr_blt'].isnull().sum()}/114")
print(f"4. Finish: {housing['garage_finish'].isnull().sum()}/114")
print(f"5. Area: {housing['garage_area'].isnull().sum()}/1")
print(f"6. Cars: {housing['garage_cars'].isnull().sum()}/1")

In [ ]:
# Rechecking the 0s
print("Furthermore, the number of 0s in these columns are:")
print(f"5. Area {len(housing[housing['garage_area']==0])}, from 113")
print(f"6. Cars {len(housing[housing['garage_cars']==0])}, from 113")

Those two investigations resolved the bulk of the NAs.  I used similar methods on the rest.  

The columns *masonry_type* and *masonry_area* were the only two columns in the dataset that refered to masonry, and thus could only be cross-checked with each other.

In [ ]:
housing['masonry_type'].value_counts(dropna = False)

In [ ]:
# Subset by the column with more NAs
masonry = housing[housing['masonry_type'].isnull()]
# See how many of the masonry_areas NAs overlap with the masonry_type NAs
print(f"Overlap: {masonry['masonry_area'].isnull().sum()}/22 total")

Once again, there was a wide discrepancy between a descriptive and a quantitative feature.

In [ ]:
print(f"Subset masonry_area 0s: {len(masonry[masonry['masonry_area']==0])}")
print(f"1213+22=1235")
print(f"Total masonry_area 0s: {len(housing[housing['masonry_area']==0])}")

Combining the 0s and the NAs in the quantitative column once again produced a sum very close to the number of NAs in the descriptive column, but not exactly the same.  There were 5 rows within the subset (i.e.,  *masonry_type* is NA) that had non-zero values in *masonry_area*, and 3 additional rows outside of the subset (i.e., *masonry_type* is populated) that had 0s in *masonry_area*.  Logically, any row with an NA in *masonry_type* should have had a 0 in *masonry_area*, and vice versa.  If masonry veneer were not present on the house, it would not take up area, and if it took up area, it was present.

In [ ]:
# Start with the subset in which all masonry_type = NA
# Filter out the NAs in area
have_area_no_type = masonry[masonry['masonry_area'].notnull()]
# Filter out the 0s in area
have_area_no_type = have_area_no_type[have_area_no_type['masonry_area']!= 0]
# Review the rows that are populated in area but not in type
have_area_no_type

In [ ]:
# What are the values?
have_area_no_type['masonry_area'].value_counts(dropna = False)

In [ ]:
# For context
housing['masonry_type'].unique()

In [ ]:
# For more context
#housing.loc[housing['Mas Vnr Area']>0, 'Mas Vnr Area'].sort_values().head(20)
# Note: I commented this code out after using it because the output is long. Single digit areas are unusual.

There were only 3 non-null values that *masonry_type* took in th dataset - brick face, brick common, or stone - and the received data dictionary listed only one more, cinder block.  This is a very short list of potential types of masonry.  A [cursory search](https://www.homequestionsanswered.com/what-is-masonry-veneer.htm) suggested that brick and stone do technically cover all the options, but there are [more subcategories](https://www.civilengineeringweb.com/2020/09/types-of-stone-masonry.html) than the ones that were given as options.  Given this fact, I found it plausible that some houses had a type of veneer that was not listed among the available choices (or, perhaps, that the assessor could not identify), and were thus erroneously coded as NA in the *masonry_type* column, despite actually having some masonry veneer.  

The fact that the 3 smallest areas (1 square foot each) in the dataset fell within this subset of 5 rows further supported this explanation.  These houses may well have had a small, ornamental piece of masonry work that did not fall neatly into any of the options for *masonry_type*.  Working under this hypothesis, I left these values in and changed the corresponding NAs in *masonry_type* to "other."  At the very least, the small number of affected values and their small *masonry_area*s limited the damage these values could cause in the model.

As for the other two rows with a populated *masonry_area* but NA *masonry_type*, these were challenging decisions.  The values were large enough that, if erroneous and if *masonry_area* proved to be a useful predictor, they could introduce a great deal of error into the predictions.  Ultimately, I relied on the assumption that such specific values (288 and 344), with multiple digits and not ending in 0 or 5, were unlikely to be accidents, mischievous reports, or misplaced statistical artifacts.  Much like in the basement analysis, I put a great deal of faith into the provenance of this dataset, and based my decisions on the simplifying assumption that government files are more likely to be accurate and complete than other sources (i.e., volunteer survey data), and therefore erred on the side of preserving the data as it was received.  For these values too, I changed the corresponding NAs in *masonry_type* to "other."

Although I once again corrected these rows by name for convenience's sake, I accounted for this situation in the hands-off cleaning code for future data.  The fact that all of the rows with a populated area but an NA for type were resolved the same way made this considerably easier.

In [ ]:
# Define a shortcut to these rows
pids = (housing['pid']==533352075) | (housing['pid']==534129230) | (
    housing['pid']==535106140) | (housing['PID']==902427140) | (housing['pid']==527166010)
#change the value
housing.loc[pids, 'masonry_type'] = 'other'
#check
housing.loc[pids, 'masonry_type']

Next, I examined the 3 rows that had 0s in *masonry_area*, but values in *masonry_type*.

In [ ]:
masonry = housing[housing['masonry_area']==0]
len(masonry) #1216
masonry = masonry[masonry['masonry_type'].notnull()]
len(masonry) #3
masonry.shape

The contrast between these two features was the only apparent problem with these rows, so I was disinclined to drop them.  Logically, there were two possible explanations for a row to list a type of masonry but no area: either the type was listed by mistake, or the masonwork was so small that it was rounded to 0.  In my opinion, either way, these houses would be better considered as having no masonry.  I cleared these entries in *masonry_type* and recoded them along with the other NAs.  This situation too was accounted for in the future-facing code.

In [ ]:
# Define a shortcut to these rows
pids = (housing['pid']==528222090) | (housing['pid']==527252070) | (
    housing['pid']==528435060)
# Change the value
housing.loc[pids, 'masonry_type'] = np.nan
# Check
housing.loc[pids, 'masonry_type']

In [ ]:
# Recode the NAs
housing['masonry_type'] = housing['masonry_type'].fillna('not-present')
housing['masonry_area'] = housing['masonry_area'].fillna(0)

In [ ]:
# Recheck the values
print(f"Type: {housing['masonry_type'].isnull().sum()} NAs, from 1238")
print(housing['masonry_type'].value_counts())
print(f"Area: {len(housing[housing['masonry_area']==0])} 0s, from 1216")

As seen above, the NAs in *masonry_type* matched the 0s in *masonry_area* post-corrections.

Of the remaining features with NAs, *pool_quality*, *fireplace_quality*, and *misc_feature* were the ones easiest to cross-check against other features: *pool_area*, *fireplaces* (the count), and *misc_value* (monetary value of miscellaneous elements in a house), respectively.  Once again, all of these features were listed in the received data dictionary as using NA to indicate houses that do not have the particular element, so I was not overly concerned about these NAs.  However, in the interest of caution and maximizing my knowledge with which to write the future-facing code, I conducted the investigation below.

In [ ]:
# Pools
housing['pool_qc'].value_counts(dropna = False)

The idea that only 9 of these houses in Ames, Iowa had pools struck me as extremely plausible.  Students at Iowa State University have [access to a pool for free](https://cyclonehealth.iastate.edu/recreation/membership), and would likely be loathe to take on the costs of owning a pool.  Non-students can also use the ISU pool for a fee, or any of [several municipal pools](https://www.cityofames.org/government/departments-divisions-i-z/parks-recreation/aquatics-pool-information-and-season-pass-fees).  As long as this value matched up with the values in *pool_area*, I saw no reason to be suspicious of it.

In [ ]:
# Subset by the NAs
pools = housing[housing['pool_qc'].isnull()]
pools.shape

In [ ]:
# How many total 0s are there?
housing['pool_area'].value_counts(dropna = False)

In [ ]:
# How many of those 0s overlap with NAs?
pools['pool_area'].value_counts(dropna = False)

The pool-related values matched up perfectly.

In [ ]:
# Fireplaces
housing['fireplace_qu'].value_counts(dropna = False)

Once again, these numbers seemed perfectly plausible.

In [ ]:
# Subset by the NAs
fireplaces = housing[housing['fireplace_qu'].isnull()]
fireplaces.shape

In [ ]:
# How many total 0s are there?
housing['fireplaces'].value_counts(dropna = False)

In [ ]:
# How many of those 0s overlap with NAs?
fireplaces['fireplaces'].value_counts(dropna = False)

The fireplace-related values matched up perfectly.

In [ ]:
# Misc. Features
housing['misc_feature'].value_counts(dropna = False)

These numbers also seemed perfectly plausible.

In [ ]:
# Subset by the NAs
miscs = housing[housing['misc_feature'].isnull()]
miscs.shape

In [ ]:
# How many total 0s are there?
len(housing[housing['misc_val']==0])

In [ ]:
# How many of those 0s overlap with NAs?
len(miscs[miscs['misc_val']==0])

The miscellaneous-related values matched up perfectly.

In [ ]:
# Recode these NAs
housing['pool_qc'] = housing['pool_qc'].fillna('not-present')
housing['fireplace_qu'] = housing['fireplace_qu'].fillna('not-present')
housing['misc_feature'] = housing['misc_feature'].fillna('not-present')

In [ ]:
# Recheck these rows
print(f" Pools: {housing['pool_qc'].isnull().sum()} NAs, from 2042")
print(f"{len(housing[housing['pool_qc']=='not-present'])} recoded values")
print(f" Fireplaces: {housing['fireplace_qu'].isnull().sum()} NAs, from 1000")
print(f"{len(housing[housing['fireplace_qu']=='not-present'])} recoded values")
print(f" Pool: {housing['misc_feature'].isnull().sum()} NAs, from 1986")
print(f"{len(housing[housing['misc_feature']=='not-present'])} recoded values")

The features *fence* and *alley* were also listed in the received data dictionary as using NA to indicate houses that did not have these elements, and there were no obvious candidates to cross-check them with.  With no other option, I am assumed that these NAs were all being used correctly, and recoded them to make future dummy coding smoother.  

In [ ]:
# Fences
print(housing['fence'].value_counts(dropna = False))
# Alleys
housing['alley'].value_counts(dropna = False)

Thankfully, these numbers seemed plausible.  There would not have been anything to do if they did not.

In [ ]:
# Recode the NAs
housing['fence'] = housing['fence'].fillna('not-present')
housing['alley'] = housing['alley'].fillna('not-present')

In [ ]:
# Recheck
print(f" Fences: {housing['fence'].isnull().sum()} NAs, from 1651")
print(f"{len(housing[housing['fence']=='not-present'])} recoded values")
print(f" Alleys: {housing['alley'].isnull().sum()} NAs, from 1911")
print(f"{len(housing[housing['alley']=='not-present'])} recoded values")

Finally, only *lot_frontage* remained.  This feature, unlike most of the others, did not list any purposeful use of NAs, despite having one of the highest numbers of them (*n*=330).  There were no NAs or 0s in *lot_area*, and how a lot could have no frontage (defined as "[Linear feet of street connected to property](https://www.kaggle.com/competitions/318-ames-competition/data)") defied my comprehension.  The only possibility I could come up with is that these lots had no frontage to a road because they were instead accessed by alleys.  Below, I checked this.

In [ ]:
# Confirm that it's not a wider problem with lot dimensions
print(f"NAs in lot_area: {housing['lot_area'].isnull().sum()}")
print(f"0s in lot_area: {len(housing[housing['lot_area']==0])}")

In [ ]:
# Subset by non-NAs
alleys = housing[housing['alley']!='not-present'] #houses must HAVE an alley to be accessed by alley
alleys.shape

My hypothesis about alley access seemed dubious at best.  There were more than double the number of houses with an NA in *lot_frontage* than there were with alleys.

In [ ]:
# Lot_frontage NAs and 0s
print(f"lot_frontage 0s: {len(housing[housing['lot_frontage']==0])}")  #good
print(f"lot_frontage NAs overall: {housing['lot_frontage'].isnull().sum()}")
#overlap
print(f"lot_frontage NAs in subset: {alleys['lot_frontage'].isnull().sum()}")

At best, my hypothesis could explain 8 rows.  I inspected these rows in hopes of gaining further insight, but was not optimistic.

In [ ]:
frontage = alleys[alleys['lot_frontage'].isnull()]
print(frontage.shape)
frontage

None of these rows were remarkable in any way, forcing me to conclude that the NAs in *lot_frontage* were truly missing data.  There should have been a value there, but there is not.

Although imputation always carries a risk of introducing error, dropping 330 out of 2051 rows (16%) would have been a dramatic loss, and leaving them as NA would have effectively dropped them from the model anyway - if the model could even be fit at all with them present.  I decided imputation was the only path forward for these values.  Because imputation often requires the use of statistics gathered from the dataset itself, I delayed this process until after partitioning the data for training and testing.  I did, however, explore some summary statistics for *lot_frontage* and related features to get an idea of what the best imputation strategy might be.  These are displayed below.

In [ ]:
# Redefine subset to be more useful
frontage = housing[housing['lot_frontage'].isnull()] 
print(frontage.shape)
housing['lot_frontage'].describe()

In [ ]:
plt.figure(figsize = (12, 6))
plt.hist(housing['lot_frontage'], color = 'purple', bins = 30);
plt.title('Distribution of Extant Lot Frontage Values', size = 30)
plt.xlabel('Lot Frontage Values', size = 24)
plt.xticks(size = 16)
plt.axline(xy1=(housing['lot_frontage'].median(), 0),
           xy2=(housing['lot_frontage'].median(), 2), color = 'yellow')
plt.ylabel('Frequency', size = 24)
plt.yticks(size = 16);
plt.tight_layout()
plt.savefig(fname = 'lot-frontage-histogram.png')

The descriptive statistics for *lot_frontage* indicated that the mean and median were very close together, which can be indicative of a normal distribution.  I plotted the distribution in hopes of confirming this, but instead discovered a severe right skew.  The distribution up to about a value of 100 was close enough to normal to be sufficient in most cases, but beyond that, a long tail of values extends past 300.  Of course, a normal distribution is nott necessary for imputation, but it would have been a positive sign that a measure of central tendency could be safely imputed without causing too many problems.

Because of the extreme skew, I became inclined to impute the mode (which also happened to be the median) rather the mean.  Before doing that though, I wanted to make sure it would not create absurdities in the context of other measurements in these rows.  In particular, I wanted to make sure that setting all of these *lot_frontage* values to about 68 feet would not make their *lot_area* values mathematically impossible. To do this, I needed some summary statistics about *lot_area* within the subset of rows for which *lot_frontage* is NA.

In [ ]:
frontage['lot_area'].describe()

In [ ]:
1533/68

The minimum lot area represented in these rows was 1533 square feet.  If that were a rectangular lot, with one side adjacent to a road, with 68 feet of frontage, the other side would have had to be 22.54 feet.  That would be a tight fit for sure, but not impossible.  With a positive, double-digit number possible, I decided to impute these values when the time came.

The last thing to investigate were a handful of columns that had 0s listed as their minimum values, despite that not making sense.  As discussed previously, the fact that this dataset came from a government record led me to assume that it was compiled with more care and diligence than average survey data, and therefore err on the side of leaving it as it was wherever possible.  Therefore, despite it being very weird, I decided to assume that the 0s in the columns for above-ground kitchens (*n*=2) and bedrooms (*n*=5) were correct.  A kitchen or a bedroom would have been a very big thing for an assessor to miss, and these values were populated with 0s, not NAs - implying that someone typed that in on purpose.  Additionally, the received data dictionary specified that the bedroom column referred to *above-ground* bedrooms, and the original column name for kitchens, *"Kitchen AbvGr,"* implied the same.  No information was offered about any potential below-ground kitchens or bedrooms, meaning that 0s in these columns did not necessarily imply that a house had no kitchens or bedrooms at all.  Further support for this interpretation came from checking the basement exposure ratings of these houses.

In [ ]:
# Isolate houses with no above-ground kitchens
no_kitch = housing[housing['kitchen_abvgr']==0]
# Look at the exposures of their basements
no_kitch['basemt_exposure'].value_counts(dropna = False)

In [ ]:
# Isolate houses with no above-ground bedrooms
no_beds = housing[housing['bedroom_abvgr']==0]
# Look at the exposures of their basements
no_beds['basemt_exposure'].value_counts(dropna = False)

All of these houses had at least average exposure, and most of them had "good" exposure, meaning it is entirely possible that their kitchens and bedrooms, despite technically being in the basement, had relatively normal windows, visibility, and access to the outdoors, and did not *feel* subterranean.  (And besides, Ames is famously [cyclone country](https://cyclones.com/).)

With that resolved, I turned my attention to the bathrooms.  It is perfectly normal for houses to have no half bathrooms, so these 0s were of no concern.  It is not very normal for houses to have no above-ground bathrooms, but much like the kitchens and bedrooms, it is plausible that they were in the basement.  However, unlike the kitchens and bedrooms, the dataset included a column for basement bathrooms.  Therefore, I checked if any houses had 0s in both their above- and below-ground bathroom columns.

In [ ]:
# Create a mask
no_full_baths = (housing['basemt_full_bath']==0)&(housing['full_bath']==0)
# View the rows
housing[no_full_baths]

The contents surprised me.  It referred to single a townhouse built in 2006, meaning it must surely have had a full bathroom somewhere.  Very old, very rural, or otherwise very unusual houses [may sometimes](https://shunshelter.com/article/can-you-sell-a-house-without-a-bathroom) be allowed to sell without a full bathroom, but a normal construction in the 21st century would surely not.  However, depsite this egregious omission, the house listed two half bathrooms each in the basement and on the main floor.  As likely as it seems that there was some kind of error here, it would have been nigh impossible to pinpoint where and what it was.  Changing one of the 0s to represent a full bathroom, despite surely being correct, would have altered this house's representation in the model to be a 1400 square foot house with *5 bathrooms* that sold for nearly \$200k.  Although this row undoubtedly contains some inaccuracies, I judged that attempting to correct it would likely only introduce more error.

As weird and unlikely as they were, the 0s for kitchens, bedrooms, and bathrooms were allowed to stay.

After all of that, at long last, all NAs were resolved.  Many of these values did not represent a problem at all, but rather were just an inconvenient way of representing the lack of something.  A handful did require a judgment call from me, but not in quantities or extents that raise too much alarm in my mind for the validity of the eventual model.  Only the column *lot_frontage* presented a true dilemma.  Far too many of these values were missing to comfortably drop the rows, nor was it appealing to drop this feature, that well may influence how much buyers are willing to pay for the property.  I decided to impute the mode for these NAs, but I will have to proceed cautiously about using this feature in the model.

In [ ]:
# Check that they're all gone
housing.isnull().sum().sum()

The reproducible, future-facing code to clean new data, based on what I learned and did here, can be found in the main report.